In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("moloco_data.csv")

In [6]:
data.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [7]:
data.shape

(3553, 4)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ts          3553 non-null   object
 1   user_id     3553 non-null   object
 2   country_id  3553 non-null   object
 3   site_id     3553 non-null   object
dtypes: object(4)
memory usage: 111.2+ KB


### Question #1

Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?


In [8]:
data_bdv=data[data['country_id']=="BDV"]

In [11]:
#Check the numpber of rows
data_bdv.shape[0]

844

In [14]:
# Group by site_id and find number of unique users by 
data_bdv.groupby("site_id").agg({"user_id":"nunique"}).sort_values('user_id', ascending=False)

,user_id
site_id,
5NPAU,544
N0OTG,90
3POLC,2


site_id=5NPAU has the largest number of unique users

### Question #2

Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [22]:
data['ts'] = pd.to_datetime(data['ts'])

In [26]:
# Create a dataframe with ts in the given range
data_day=data[(data['ts']>"2019-02-03 00:00:00")&(data['ts']<="2019-02-04 23:59:59")]

In [31]:
# Get a cross tab with the couts of user visits per each site
site_users = pd.crosstab(data_day['user_id'], data_day['site_id'])

In [37]:

site_users

site_id,3POLC,5NPAU,GVOFK,JSUUP,N0OTG,QGO3G
user_id,,,,,,
LC00C3,0,1,0,0,0,0
LC01C3,0,1,0,0,0,0
LC06C3,0,0,1,0,25,0
LC0C3E,0,0,0,0,1,0
LC13C3,0,1,0,0,0,0
...,...,...,...,...,...,...
LCDFC3,0,0,0,0,1,0
LCE2C3,0,3,0,0,0,0
LCE4C3,0,0,0,0,1,0


In [43]:
site_names = data_day['site_id'].unique()

In [44]:
site_names

array(['3POLC', 'N0OTG', 'QGO3G', '5NPAU', 'GVOFK', 'JSUUP'], dtype=object)

In [75]:
# For each site find rows (users) where the count of visits is >10
for site in site_names:
    if len(site_users[site_users[site]>10].index.values) != 0:
        for user in site_users[site_users[site]>10].index.values:
                print((user, site, site_users[site][user]))




('LC3C7E', '3POLC', 15)
('LC06C3', 'N0OTG', 25)
('LC3A59', 'N0OTG', 26)
('LC3C9D', 'N0OTG', 17)


In [77]:
# Checking:
for site in site_names:
    if len(site_users[site_users[site]>10].index.values) != 0:
        print(site)
        print(site_users[site_users[site]>10])

3POLC
site_id  3POLC  5NPAU  GVOFK  JSUUP  N0OTG  QGO3G
user_id                                          
LC3C7E      15      0      0      0      0      0
N0OTG
site_id  3POLC  5NPAU  GVOFK  JSUUP  N0OTG  QGO3G
user_id                                          
LC06C3       0      0      1      0     25      0
LC3A59       0      0      0      0     26      0
LC3C9D       0      0      1      0     17      0


### Question #3

For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [102]:
data_last = data.groupby('user_id').last().reset_index()

In [112]:
# Check for user LC3561
data_last[data_last['user_id']=="LC3561"]

,user_id,ts,country_id,site_id
581,LC3561,2019-02-07 23:44:34,TL6,N0OTG


In [116]:
top5= data_last.groupby('site_id').agg({"user_id":"nunique"}).sort_values('user_id', ascending=False)

In [121]:
top5

,user_id
site_id,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


In [129]:
top5[:3]

,user_id
site_id,
5NPAU,992
N0OTG,561
QGO3G,289


### Question 4


For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [131]:
data_first_last = data.groupby('user_id').agg(['first','last']).reset_index()

In [138]:
data_first_last

user_id                  ts                     country_id      site_id  \
                           first                last      first last   first   
0     LC00C3 2019-02-03 18:52:50 2019-02-03 18:52:50        QLT  QLT   5NPAU   
1     LC01C3 2019-02-04 11:35:10 2019-02-04 11:35:10        QLT  QLT   5NPAU   
2     LC05C3 2019-02-02 14:14:44 2019-02-02 14:14:44        BDV  BDV   5NPAU   
3     LC06C3 2019-02-01 22:49:39 2019-02-07 01:16:12        TL6  TL6   N0OTG   
4     LC07C3 2019-02-05 19:06:42 2019-02-05 19:06:42        BDV  BDV   5NPAU   
...      ...                 ...                 ...        ...  ...     ...   
1911  LCFC3B 2019-02-05 04:53:03 2019-02-05 04:53:03        TL6  TL6   N0OTG   
1912  LCFC3D 2019-02-01 18:59:50 2019-02-01 18:59:50        TL6  TL6   N0OTG   
1913  LCFC3E 2019-02-01 20:49:08 2019-02-01 20:49:13        BDV  BDV   5NPAU   
1914  LCFEC3 2019-02-02 01:19:49 2019-02-07 06:23:59        TL6  HVQ   N0OTG   
1915  LCFFC3 2019-02-02 22:36:23 2019-02-05 03:31:17        XA7  XA7   N0OTG   

             
       last  
0     5NPAU  
1     5NPAU  
2     5NPAU  
3     N0OTG  
4     5NPAU  
...     ...  
1911  N0OTG  
1912  N0OTG  
1913  5NPAU  
1914  3POLC  
1915  N0OTG  

[1916 rows x 7 columns]

In [140]:
result = data_first_last[data_first_last['site_id']['first']==data_first_last['site_id']['last']]

In [142]:
result.head()

user_id                  ts                     country_id      site_id  \
                        first                last      first last   first   
0  LC00C3 2019-02-03 18:52:50 2019-02-03 18:52:50        QLT  QLT   5NPAU   
1  LC01C3 2019-02-04 11:35:10 2019-02-04 11:35:10        QLT  QLT   5NPAU   
2  LC05C3 2019-02-02 14:14:44 2019-02-02 14:14:44        BDV  BDV   5NPAU   
3  LC06C3 2019-02-01 22:49:39 2019-02-07 01:16:12        TL6  TL6   N0OTG   
4  LC07C3 2019-02-05 19:06:42 2019-02-05 19:06:42        BDV  BDV   5NPAU   

          
    last  
0  5NPAU  
1  5NPAU  
2  5NPAU  
3  N0OTG  
4  5NPAU

In [143]:
print("the number of users whose first/last visits are to the same website:",result.shape[0])

the number of users whose first/last visits are to the same website: 1670
